# Plot the planned/micro-manip/ephys trajectories for each mouse

In [1]:
import unityneuro.render as urn
from one.api import ONE
one = ONE(base_url='https://alyx.internationalbrainlab.org')
import iblatlas.atlas as atlas
NeedlesAtlas = atlas.NeedlesAtlas(25)
CCFAtlas = atlas.AllenAtlas(25)

import random
r = lambda: random.randint(0,255)
randHexColor = lambda : '#%02X%02X%02X' % (r(),r(),r())

TEXT_X = -600
TEXT_Y_INC = 30

In [2]:
def getCoords(ins, coord_transform):
    entry_coords = coord_transform.xyz2ccf(ins.entry, mode='wrap')
    tip_coords = coord_transform.xyz2ccf(ins.tip, mode='wrap')
    angles = [ins.phi, ins.theta, ins.beta]
    # depth = vec_dist(entry_coords, tip_coords)
    return (entry_coords, tip_coords, angles)

def traj2coords(traj, coord_transform):
    insertion = atlas.Insertion.from_dict(traj)
    return getCoords(insertion, coord_transform)

def plot_session_micromanip(mouse):
    bwm_mm = one.alyx.rest('trajectories', 'list', provenance='Micro-manipulator', subject=mouse,
                          project='ibl_neuropixel_brainwide_01', use_cache=False)

    urn.clear()

    # get all the insertions for this mouse
    urn.set_area_visibility({'8':True})
    urn.set_area_material({'8':'transparent-unlit'})
    urn.set_area_alpha({'8':0.15})
    
    probe_names = []
    probet_texts = {}
    probet_colors = {}
    probet_positions = {}

    for i, ins in enumerate(bwm_mm):
        # get the coords
        (entry, tip, angles) = traj2coords(ins, NeedlesAtlas)
        
        pname = 'mm'+str(i)
        urn.create_probes([pname])
        urn.set_probe_positions({pname:list(tip)})
        urn.set_probe_angles({pname:list(angles)})
        color = randHexColor()
        urn.set_probe_colors({pname:color})
        probe_names.append(pname)
        probet_texts[pname] = ins['session']['start_time'][0:10] + '/' + ins['probe_name']
        probet_colors[pname] = color
        probet_positions[pname] = [TEXT_X,i*TEXT_Y_INC]
        
    urn.create_text(probe_names)
    urn.set_text(probet_texts)
    urn.set_text_colors(probet_colors)
    urn.set_text_positions(probet_positions)
        
    urn.create_text(['mouse_name'])
    urn.set_text({'mouse_name':mouse})
    urn.set_text_colors({'mouse_name':'#FFA500'})

    urn.set_camera_rotation(45,0,0)
    
    return bwm_mm

In [3]:
urn.setup()

(URN) connected to server
Login sent with ID: Dan


In [4]:
bwm_mm = plot_session_micromanip('ZFM-01576')

In [5]:
def plot_session(mouse, pid):
    color_opts = {'planned':'#0000ff', 'Micro-manipulator':'#00ff00', 'Histology track':'#ff0000', 'Ephys-aligned histology track':'#ff00ff'}
    atlas_opts = {'planned':NeedlesAtlas, 'Micro-manipulator':NeedlesAtlas, 'Histology track':CCFAtlas, 'Ephys-aligned histology track':CCFAtlas}

    bwm_trajs = one.alyx.rest('trajectories', 'list', subject=mouse, probe_insertion=pid,
                          project='ibl_neuropixel_brainwide_01', use_cache=False)

    urn.clear()
    
    urn.set_area_visibility({'8':True})
    urn.set_area_material({'8':'transparent-unlit'})
    urn.set_area_alpha({'8':0.15})
    
    # group data by EIDs
    data = {}

    for traj in bwm_trajs:
        data[traj['provenance']] = traj

    info_text = data['Micro-manipulator']['datetime'][:10] + '/' + data['Micro-manipulator']['probe_name']
    urn.create_text(['eid_info'])
    urn.set_text({'eid_info':info_text})

    # create the legend

    urn.create_text(list(color_opts.keys()))
    urn.set_text({'planned':'planned', 'Micro-manipulator':'Micro-manipulator', 'Histology track':'Histology', 'Ephys-aligned histology track':'Ephys-aligned histology'})
    urn.set_text_positions({'planned':[TEXT_X,TEXT_Y_INC*1], 'Micro-manipulator':[TEXT_X,TEXT_Y_INC*2], 'Histology track':[TEXT_X,TEXT_Y_INC*3], 'Ephys-aligned histology track':[TEXT_X,TEXT_Y_INC*4]})
    urn.set_text_colors(color_opts)

    probe_names = []
    probet_texts = {}
    probet_colors = {}
    probet_positions = {}

    probe_positions = {}
    probe_angles = {}
    probe_colors = {}

    for ins_type in color_opts.keys():
        if ins_type in data.keys():
            probe_names.append(ins_type)

            (entry, tip, angles) = traj2coords(data[ins_type], atlas_opts[ins_type])

            probe_positions[ins_type] = list(tip)
            probe_angles[ins_type] = list(angles)
            probe_colors[ins_type] = color_opts[ins_type]

    urn.create_probes(probe_names)
    urn.set_probe_positions(probe_positions)
    urn.set_probe_angles(probe_angles)
    urn.set_probe_colors(probe_colors)

In [6]:
plot_session('NR_0020','dac5defe-62b8-4cc0-96cb-b9f923957c7f')

In [7]:
plot_session('ZFM-01576','6eb8be8d-d089-43c6-b2e4-a6558ca16dcf')